In [1]:
import wikipedia
import pandas as pd
from tqdm import tqdm 
import math
import numpy as np
import html
import re
from unidecode import unidecode
import pickle

In [2]:
#add in anime not from combined
#imdb and myanimelist
#mine out the plot for each anime title
#get the animelist name to look on wikipedia 

### See which ones still might need to be added in

In [3]:
df_joined = pd.read_json("/Users/justinvhuang/Downloads/2024-03-13-ETLs/metadata.json")
df_imdb = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/imdb_scraped_datav2.csv", index_col=0)
df_imdb_episode = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/imdb_anime.csv")

In [4]:
already_joined = df_joined['imdb_title_titleId'].tolist()

In [5]:
df_imdb_not_in = df_imdb[~df_imdb['imdb_id'].isin(already_joined)].drop_duplicates('title')

In [6]:
df_imdb_episode['Episode Title'] = df_imdb_episode['Episode Title'].replace(np.nan, 'none')

In [7]:
df_imdb_episode_no = df_imdb_episode[df_imdb_episode['Episode Title'] == 'none']

In [8]:
# Function to lowercase words and remove punctuation
def clean_text(text):
    return ' '.join(re.findall(r'\b\w+\b', text.lower()))

# Function to remove diacritics
def remove_diacritics(text):
    clean_text = unidecode(text)

    clean_text = html.unescape(clean_text)
    return clean_text


In [9]:
df_imdb_not_in['title'] = df_imdb_not_in['title'].apply(clean_text).apply(remove_diacritics)

In [10]:
df_imdb_episode_no['title'] = df_imdb_episode_no['Title'].apply(clean_text).apply(remove_diacritics)

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_97478/712706123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imdb_episode_no['title'] = df_imdb_episode_no['Title'].apply(clean_text).apply(remove_diacritics)


In [11]:
df_addon = df_imdb_not_in.merge(df_imdb_episode_no, on = 'title', how = 'inner')

In [12]:
df_addon['keywords'] = df_addon['keywords'].replace(np.nan, 'none')
df_addon = df_addon[df_addon['keywords'] != 'none'].drop_duplicates(subset=['title', 'imdb_id'])

In [13]:
df_joined.columns

Index(['imdb_scrape_imdb_id', 'imdb_title_attributes',
       'imdb_title_isOriginalTitle', 'imdb_title_language',
       'imdb_title_ordering', 'imdb_title_region', 'imdb_title_title',
       'imdb_title_titleId', 'imdb_title_types', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Episodes',
       'anime_Favorites', 'anime_Genres', 'anime_Image URL', 'anime_Licensors',
       'anime_Members', 'anime_Name', 'anime_Other name', 'anime_Popularity',
       'anime_Premiered', 'anime_Producers', 'anime_Rank', 'anime_Rating',
       'anime_Score', 'anime_Scored By', 'anime_Source', 'anime_Status',
       'anime_Studios', 'anime_Synopsis', 'anime_Type', 'anime_anime_id',
       'imdb_principals_tconst', 'imdb_name_basics_primaryName'],
      dtype='object')

In [14]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [15]:
df_joined[['imdb_title_title', 'anime_Aired', 'anime_Duration', 'anime_English name', 'anime_Episodes', 'anime_Favorites', 'anime_Genres', 'anime_Image URL', 'anime_Licensors', 'anime_Name', 'anime_Other name', 'anime_Popularity',
       'anime_Premiered', 'anime_Producers', 'anime_Rank', 'anime_Rating',
       'anime_Score', 'anime_Source', 'anime_Status',
       'anime_Studios', 'anime_Synopsis', 'anime_Type', 'anime_anime_id', 'imdb_name_basics_primaryName']]

,imdb_title_title,anime_Aired,anime_Duration,anime_English name,anime_Episodes,anime_Favorites,anime_Genres,anime_Image URL,anime_Licensors,anime_Name,anime_Other name,anime_Popularity,anime_Premiered,anime_Producers,anime_Rank,anime_Rating,anime_Score,anime_Source,anime_Status,anime_Studios,anime_Synopsis,anime_Type,anime_anime_id,imdb_name_basics_primaryName
0,【推しの子】,None,None,[Oshi No Ko],None,NaN,"Drama, Supernatural",None,None,"\Oshi no Ko\""""",【推しの子】,NaN,None,None,None,None,8.98,None,None,None,"In the entertainment world, celebrities often ...",None,52034.0,"Natalie Rial, Christina Marie Kelly, Takeo Ots..."
1,.hack//G.U. Returner,None,None,UNKNOWN,None,NaN,"Adventure, Drama, Fantasy",None,None,.hack//G.U. Returner,.HACK//G.U. RETURNER,NaN,None,None,None,None,6.67,None,None,None,The characters from previous .hack//G.U. Games...,None,2928.0,"Kaori Nazuka, Ayako Kawasumi, Takahiro Sakurai..."
2,.hack//G.U. Trilogy,None,None,.hack//G.U. Trilogy: Parody Mode,None,NaN,"Comedy, Fantasy, Sci-Fi",None,None,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,NaN,None,None,None,None,6.36,None,None,None,A special bonus Parody Mode added to the extra...,None,4469.0,"Hiroshi Matsuyama, Ayako Kawasumi, Saeko Hirot..."
3,.hack//GIFT,"Nov 16, 2003",26 min,.hack//Gift,1.0,19.0,"Comedy, Fantasy",https://cdn.myanimelist.net/images/anime/2/230...,Bandai Entertainment,.hack//Gift,.hack//GIFT,4517.0,UNKNOWN,CyberConnect2,8363.0,R+ - Mild Nudity,6.1,Original,Finished Airing,Bee Train,As an expression of gratitude for the heroes o...,OVA,454.0,"Isshin Chiba, Shin'ya Kawatsura, Sayaka Aida, ..."
4,.hack//SIGN,None,None,.hack//Sign,None,NaN,"Adventure, Fantasy, Mystery",None,None,.hack//Sign,.hack//SIGN,NaN,None,None,None,None,6.95,None,None,None,"A young wavemaster, only known by the alias of...",None,48.0,"Amanda Winn Lee, Kaori Nazuka, Donna Rawlins, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,I've Always Liked You,None,None,I've Always Liked You,None,NaN,Romance,None,None,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,NaN,None,None,None,None,7.22,None,None,None,Love is blooming at Sakuragaoka High School. N...,None,31245.0,"Kana Asumi, Gero, Yoshimi Narita, Momo Asakura..."
5179,エクスドライバー Nina&Rei Danger Zone,None,None,UNKNOWN,None,NaN,"Action, Sci-Fi",None,None,eX-Driver: Nina & Rei Danger Zone,エクスドライバー Nina&Rei Danger Zone,NaN,None,None,None,None,6.39,None,None,None,The story takes place on a world 100 years in ...,None,1733.0,"Wataru Tanaka, Michiko Neya, Mami Kingetsu, Kô..."
5180,スクライド,None,None,s-CRY-ed,None,NaN,"Action, Adventure, Drama, Sci-Fi",None,None,s.CRY.ed,スクライド,NaN,None,None,None,None,7.36,None,None,None,A strange environmental phenomenon 22 years ag...,None,168.0,"Lex Lang, Jamieson Price, Dave Wittenberg, Cri..."
5181,xxxHOLiC,"Apr 7, 2006 to Sep 29, 2006",24 min per ep,xxxHOLiC,24.0,4392.0,"Comedy, Drama, Mystery, Supernatural",https://cdn.myanimelist.net/images/anime/5/218...,Funimation,xxxHOLiC,×××HOLiC（ホリック）,714.0,spring 2006,"TBS, BMG Japan, Kids Station, Kodansha, Movic",614.0,PG-13 - Teens 13 or older,7.98,Manga,Finished Airing,Production I.G,Kimihiro Watanuki can see spirits and other as...,TV,861.0,"Shizuka Itou, Hisayo Mochizuki, Colleen Clinke..."


In [16]:
df_joined['anime_anime_id'] = df_joined['anime_anime_id'].replace(np.nan, 'none')

In [17]:
df_joined = df_joined[df_joined['anime_anime_id'] != 'none']

In [18]:
myanime_list = df_joined['anime_anime_id'].apply(lambda x: int(x)).tolist()

In [19]:
file_path = "my_animelist.pkl"

# Open the file in binary write mode and save the list using pickle.dump()
with open(file_path, 'wb') as f:
    pickle.dump(myanime_list, f)

#### Wikipedia Information 

In [20]:
wiki_pull_df = pd.concat([df_joined[['imdb_title_titleId', 'anime_Other name']].rename(columns={'imdb_title_titleId': 'imdb_id', 'anime_Other name': 'Title'}), df_addon[['Title', 'imdb_id']]],ignore_index=True)

In [25]:
# Function to get Wikipedia content for a given title
def get_wikipedia_content(title):
    import time

    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        # If the title is ambiguous, you can handle it as per your requirement
        return f"Ambiguous title: {title}. Options: {e.options}"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for: {title}"
    except wikipedia.exceptions.HTTPTimeoutError as e:
        return f"Timeout error occurred: {str(e)}"
    except wikipedia.exceptions.WikipediaException as e:
        if "HTTP 443" in str(e):
            print("Received HTTP 443 error. Pausing for 2 minutes...")
            time.sleep(120)  # Pause for 2 minutes (120 seconds)
            print("Resuming...")
            return get_wikipedia_content(title)  # Retry
        else:
            return f"Error occurred: {str(e)}"
    except Exception as e:
        return f"Error occurred: {str(e)}"
    
def fetch_wikipedia_content(df, get_wikipedia_content_func):
    # Dictionary to store data
    data_dict = {}

    # Loop through each title in the DataFrame with tqdm for progress tracking
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Fetching Wikipedia Content"):
        imdb_id = row['imdb_id']
        title = row['Title']

        # Get Wikipedia content for the title
        content = get_wikipedia_content_func(title)

        # Add data to dictionary
        if imdb_id not in data_dict:
            data_dict[imdb_id] = {'Title': title, 'Content': content}
        else:
            print(f"Duplicate IMDb ID found: {imdb_id}")

    return data_dict

In [30]:
pull1 = wiki_pull_df.iloc[0:1500,:]
pull2 = wiki_pull_df.iloc[1500:3000,:]
pull3 = wiki_pull_df.iloc[3000:4500,:]
pull4 = wiki_pull_df.iloc[4500:,:]

In [27]:
fetch1 = fetch_wikipedia_content(pull1, get_wikipedia_content)

Fetching Wikipedia Content:   1%|▏         | 19/1500 [00:19<23:20,  1.06it/s]

Duplicate IMDb ID found: tt0189238


Fetching Wikipedia Content:   2%|▏         | 37/1500 [00:36<18:35,  1.31it/s]

Duplicate IMDb ID found: tt0168283


Fetching Wikipedia Content:   3%|▎         | 45/1500 [00:44<21:20,  1.14it/s]

...

Fetching Wikipedia Content:  96%|█████████▋| 1444/1500 [24:03<00:52,  1.08it/s]

Duplicate IMDb ID found: tt0085092


Fetching Wikipedia Content:  98%|█████████▊| 1468/1500 [24:28<00:29,  1.07it/s]

Duplicate IMDb ID found: tt0429439


Fetching Wikipedia Content: 100%|██████████| 1500/1500 [25:02<00:00,  1.00s/it]


In [28]:
fetch2 = fetch_wikipedia_content(pull2, get_wikipedia_content)

Fetching Wikipedia Content:   3%|▎         | 52/1500 [00:47<16:20,  1.48it/s]

Duplicate IMDb ID found: tt2061527


Fetching Wikipedia Content:   4%|▍         | 63/1500 [00:58<22:33,  1.06it/s]/Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Fetching Wikipedia Content:   9%|▉         | 141/1500 [02:09<18:30,  1.22it/s]

Duplicate IMDb ID found: tt4760416


Fetching Wikipedia Content:  10%|█         | 150/1500 [02:17<17:55,  1.26it/s]

...

Fetching Wikipedia Content: 100%|█████████▉| 1494/1500 [25:21<00:04,  1.24it/s]

Duplicate IMDb ID found: tt2011519


Fetching Wikipedia Content: 100%|█████████▉| 1496/1500 [25:23<00:03,  1.23it/s]

Duplicate IMDb ID found: tt7790776


Fetching Wikipedia Content: 100%|██████████| 1500/1500 [25:26<00:00,  1.02s/it]


In [31]:
fetch3 = fetch_wikipedia_content(pull3, get_wikipedia_content)

Fetching Wikipedia Content:   0%|          | 5/1500 [00:03<15:07,  1.65it/s]

Duplicate IMDb ID found: tt0426719


Fetching Wikipedia Content:   2%|▏         | 31/1500 [00:15<13:26,  1.82it/s]

Duplicate IMDb ID found: tt0810056


Fetching Wikipedia Content:   2%|▏         | 32/1500 [00:16<13:14,  1.85it/s]

...

Fetching Wikipedia Content:  98%|█████████▊| 1476/1500 [13:32<00:15,  1.51it/s]

Duplicate IMDb ID found: tt4219258


Fetching Wikipedia Content:  99%|█████████▉| 1482/1500 [13:35<00:10,  1.64it/s]

Duplicate IMDb ID found: tt1738090


Fetching Wikipedia Content: 100%|██████████| 1500/1500 [13:46<00:00,  1.82it/s]


In [32]:
fetch4 = fetch_wikipedia_content(pull4, get_wikipedia_content)

Fetching Wikipedia Content:   0%|          | 2/1339 [00:00<05:22,  4.15it/s]

Duplicate IMDb ID found: tt3646574


Fetching Wikipedia Content:   2%|▏         | 24/1339 [00:10<10:50,  2.02it/s]

Duplicate IMDb ID found: tt3398976


Fetching Wikipedia Content:   3%|▎         | 46/1339 [00:22<11:57,  1.80it/s]

...

Fetching Wikipedia Content:  50%|█████     | 675/1339 [09:24<08:55,  1.24it/s]

Duplicate IMDb ID found: tt9054354


Fetching Wikipedia Content:  50%|█████     | 676/1339 [09:25<09:15,  1.19it/s]

Duplicate IMDb ID found: tt9054354


Fetching Wikipedia Content: 100%|██████████| 1339/1339 [20:19<00:00,  1.10it/s]


In [38]:
# Combine dictionaries
combined_dict = {}
combined_dict.update(fetch1)
combined_dict.update(fetch2)
combined_dict.update(fetch3)
combined_dict.update(fetch4)

In [41]:
df = pd.DataFrame(combined_dict)

In [45]:
df_wiki = df.T.reset_index()

In [47]:
df_wiki.loc[5155, 'Content']

'No Wikipedia page found for: Kitsune no homerun-ô'

In [50]:
filtered_df_wiki = df_wiki[~df_wiki['Content'].str.startswith('No Wikipedia page found for:')]


In [70]:
print(filtered_df_wiki.loc[200, 'Content'])

Super Electromagnetic Machine Voltes V (Japanese: 超電磁マシーン ボルテスV（ファイブ）, Hepburn: Chōdenji Mashīn Borutesu Faibu), popularly known as simply Voltes V  (pronounced as "Voltes Five") is a Japanese anime television series produced by Toei Company and animated by Nippon Sunrise (now known as Bandai Namco Filmworks and formerly known as Soeisha). It is the second installment of the Robot Romance Trilogy, which also includes Chōdenji Robo Combattler V and Tōshō Daimos. It is directed by Tadao Nagahama and produced by Yoshiyuki Tomino. It aired on TV Asahi and its affiliates from June 1977 to March 1978.
Voltes V grew into a cultural phenomenon in the Philippines since its airing in the country in 1978, and had achieved popularity as well in Indonesia and Cuba. A Philippine live-action adaptation, Voltes V: Legacy, premiered on GMA Network in May 2023.


== Plot ==
One day, a massive horde of flying saucers appears out of the depths of space and launches an invasion of the planet Earth. Despite

In [81]:
missing_df = wiki_pull_df[~wiki_pull_df['imdb_id'].isin(filtered_df_wiki['index'].tolist())]

In [95]:
pull5 = df_joined[df_joined['imdb_scrape_imdb_id'].isin(missing_df['imdb_id'].tolist())][['imdb_scrape_imdb_id','anime_Name']].rename(columns = {"imdb_scrape_imdb_id": "imdb_id", "anime_Name": "Title"})

In [96]:
fetch5 = fetch_wikipedia_content(pull5, get_wikipedia_content)

Fetching Wikipedia Content:   8%|▊         | 37/454 [00:31<05:01,  1.38it/s]/Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/justinvhuang/miniconda3/envs/spacy/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Fetching Wikipedia Content:  19%|█▉        | 88/454 [01:14<04:56,  1.23it/s]

Duplicate IMDb ID found: tt0162467


Fetching Wikipedia Content:  20%|█▉        | 89/454 [01:15<05:06,  1.19it/s]

Duplicate IMDb ID found: tt0162467


Fetching Wikipedia Content:  20%|██        | 91/454 [01:17<05:32,  1.09it/s]

...

Fetching Wikipedia Content:  95%|█████████▍| 430/454 [06:06<00:20,  1.19it/s]

Duplicate IMDb ID found: tt8372010


Fetching Wikipedia Content:  96%|█████████▌| 434/454 [06:09<00:14,  1.35it/s]

Duplicate IMDb ID found: tt3222236


Fetching Wikipedia Content: 100%|██████████| 454/454 [06:26<00:00,  1.18it/s]


In [98]:
# Combine dictionaries
combined_dict = {}
combined_dict.update(fetch1)
combined_dict.update(fetch2)
combined_dict.update(fetch3)
combined_dict.update(fetch4)
combined_dict.update(fetch5)


In [99]:
df_wiki2 = pd.DataFrame(combined_dict).T

In [101]:
filtered_df_wiki2 = df_wiki2[~df_wiki2['Content'].str.startswith('No Wikipedia page found for:')]

In [106]:
filtered_df_wiki2 = filtered_df_wiki2.reset_index()

In [107]:
missing_df = wiki_pull_df[~wiki_pull_df['imdb_id'].isin(filtered_df_wiki2['index'].tolist())]

In [124]:
missing_df

,imdb_id,Title
87,tt8230420,暗殺教室 episode:0 出会いの時間
127,tt0381895,あずまんが大王（劇場短編）
154,tt8160050,BE-BOY KIDNAPP'N IDOL
170,tt9843368,バイオハザード4 Incubate
190,tt5249462,僕だけがいない街
...,...,...
5824,tt1933701,Babel nisei
5832,tt0293728,King Kong
5835,tt0056739,Astro Boy
5836,tt0142908,Tetsujin nijûhachi-go


In [119]:
df_joined[df_joined['imdb_title_title'] == 'One Piece']

,imdb_scrape_imdb_id,imdb_title_attributes,imdb_title_isOriginalTitle,imdb_title_language,imdb_title_ordering,imdb_title_region,imdb_title_title,imdb_title_titleId,imdb_title_types,anime_Aired,anime_Duration,anime_English name,anime_Episodes,anime_Favorites,anime_Genres,anime_Image URL,anime_Licensors,anime_Members,anime_Name,anime_Other name,anime_Popularity,anime_Premiered,anime_Producers,anime_Rank,anime_Rating,anime_Score,anime_Scored By,anime_Source,anime_Status,anime_Studios,anime_Synopsis,anime_Type,anime_anime_id,imdb_principals_tconst,imdb_name_basics_primaryName
1131,tt0388629,\\N,0,\\N,7,US,One Piece,tt0388629,imdbDisplay,None,None,One Piece,None,NaN,"Action, Adventure, Fantasy",None,None,NaN,One Piece,ONE PIECE,NaN,None,None,None,None,8.69,None,None,None,None,"Gol D. Roger was known as the \Pirate King,\"" ...",None,21.0,tt0388629,"Akemi Okamura, Laurent Vernin, Colleen Clinken..."


In [125]:
missing_df[missing_df['imdb_id'] == 'tt0388629']

,imdb_id,Title
1131,tt0388629,ONE PIECE
1134,tt0388629,ONE PIECE


In [133]:
missing_df.drop_duplicates(subset = 'imdb_id')

,imdb_id,Title
87,tt8230420,暗殺教室 episode:0 出会いの時間
127,tt0381895,あずまんが大王（劇場短編）
154,tt8160050,BE-BOY KIDNAPP'N IDOL
170,tt9843368,バイオハザード4 Incubate
190,tt5249462,僕だけがいない街
...,...,...
5824,tt1933701,Babel nisei
5832,tt0293728,King Kong
5835,tt0056739,Astro Boy
5836,tt0142908,Tetsujin nijûhachi-go


In [165]:
test_text = filtered_df_wiki2['Content'][223]

In [166]:
# Define the regular expression pattern
pattern = r'^.*?\n\n\n(.*?)\n\n\n'

# Find the match
match = re.search(pattern, test_text, re.DOTALL)

In [167]:
print(match.group(0))

Cat's Eye (Japanese: キャッツ♥アイ, Hepburn: Kyattsu Ai, stylized as CAT'S♥EYE) is a Japanese manga series written and illustrated by Tsukasa Hojo. It was serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump from 1981 to 1985, with its chapters collected into 18 tankōbon. The story follows the adventures of the three Kisugi sisters — Hitomi, Rui and Ai, who are formidable art thieves trying to collect all the works belonging to their missing father.
The manga was made into a televised anime series of the same name originally broadcast in 1983 to 1984 on Nippon TV, with a second season ending in 1985. It has also received two live-action adaptations; a TV movie in 1988 and a theatrical film in 1997. A crossover ONA with Lupin III that TMS Entertainment produced, premiered on Amazon Prime Video in 2023.
Cat's Eye is one of Weekly Shōnen Jump's best-selling manga series of all time, with over 20 million copies sold, making it one of the best-selling manga series.


== Plot ==
Hitom

In [174]:
filtered_df_wiki2['match'] = filtered_df_wiki2['Content'].apply(lambda x: re.search(pattern, x, re.DOTALL))

In [181]:
filtered_df_wiki2['plot'] = filtered_df_wiki2['match'].apply(lambda x: x.group(0)if x else "None")

In [197]:
join1 = filtered_df_wiki2[filtered_df_wiki2['plot'] != 'None'][['index','plot']]

In [222]:
fin_df  = df_joined.merge(join1, left_on = 'imdb_title_titleId', right_on = 'index', how = 'left')

In [223]:
fin_df.columns

Index(['imdb_scrape_imdb_id', 'imdb_title_attributes',
       'imdb_title_isOriginalTitle', 'imdb_title_language',
       'imdb_title_ordering', 'imdb_title_region', 'imdb_title_title',
       'imdb_title_titleId', 'imdb_title_types', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Episodes',
       'anime_Favorites', 'anime_Genres', 'anime_Image URL', 'anime_Licensors',
       'anime_Members', 'anime_Name', 'anime_Other name', 'anime_Popularity',
       'anime_Premiered', 'anime_Producers', 'anime_Rank', 'anime_Rating',
       'anime_Score', 'anime_Scored By', 'anime_Source', 'anime_Status',
       'anime_Studios', 'anime_Synopsis', 'anime_Type', 'anime_anime_id',
       'imdb_principals_tconst', 'imdb_name_basics_primaryName', 'index',
       'plot'],
      dtype='object')

In [224]:
fin_df = fin_df[['imdb_title_title',
       'imdb_title_titleId', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Genres', 'anime_Image URL', 'anime_Licensors',
       'anime_Members', 'anime_Name', 'anime_Other name', 'anime_Producers', 'anime_Rating',
       'anime_Score', 
       'anime_Studios', 'anime_Synopsis', 'anime_anime_id', 'imdb_name_basics_primaryName', 
       'plot']].reset_index(drop = True)

In [225]:
fin_df2 = df_addon.merge(join1, left_on = 'imdb_id', right_on = 'index', how = 'left')

In [266]:
final_df = fin_df.merge(df_imdb[['keywords', 'imdb_id', 'image', 'description']].reset_index(drop = True), left_on = 'imdb_title_titleId', right_on = 'imdb_id', how = 'left').drop_duplicates(subset =['imdb_title_title', 'anime_anime_id']).reset_index(drop = True)

In [269]:
final_df['text'] = final_df['plot'].fillna('description')

In [278]:
final_df

,imdb_title_title,imdb_title_titleId,anime_Aired,anime_Duration,anime_English name,anime_Genres,anime_Image URL,anime_Licensors,anime_Members,anime_Name,anime_Other name,anime_Producers,anime_Rating,anime_Score,anime_Studios,anime_Synopsis,anime_anime_id,imdb_name_basics_primaryName,plot,keywords,imdb_id,image,description,text
0,【推しの子】,tt21030032,None,None,[Oshi No Ko],"Drama, Supernatural",None,None,NaN,"\Oshi no Ko\""""",【推しの子】,None,None,8.98,None,"In the entertainment world, celebrities often ...",52034.0,"Natalie Rial, Christina Marie Kelly, Takeo Ots...","Oshi no Ko (【推しの子】, ""My Favorite Idol"" or ""The...","idol,anime animation,based on manga,singer,per...",tt21030032,https://m.media-amazon.com/images/M/MV5BYWFkZD...,A doctor and his recently-deceased patient are...,"Oshi no Ko (【推しの子】, ""My Favorite Idol"" or ""The..."
1,.hack//G.U. Returner,tt1343075,None,None,UNKNOWN,"Adventure, Drama, Fantasy",None,None,NaN,.hack//G.U. Returner,.HACK//G.U. RETURNER,None,None,6.67,None,The characters from previous .hack//G.U. Games...,2928.0,"Kaori Nazuka, Ayako Kawasumi, Takahiro Sakurai...",.hack//G.U. is a series of single-player actio...,.hack,tt1343075,https://m.media-amazon.com/images/M/MV5BYjViMD...,The characters from previous .hack//G.U. Games...,.hack//G.U. is a series of single-player actio...
2,.hack//G.U. Trilogy,tt1164545,None,None,.hack//G.U. Trilogy: Parody Mode,"Comedy, Fantasy, Sci-Fi",None,None,NaN,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,None,None,6.36,None,A special bonus Parody Mode added to the extra...,4469.0,"Hiroshi Matsuyama, Ayako Kawasumi, Saeko Hirot...",.hack//G.U. Trilogy (Japanese: ドットハック G.U. Tri...,.hack,tt1164545,https://m.media-amazon.com/images/M/MV5BODU4Nz...,After one of his friends falls into a coma pla...,.hack//G.U. Trilogy (Japanese: ドットハック G.U. Tri...
3,.hack//GIFT,tt0823406,"Nov 16, 2003",26 min,.hack//Gift,"Comedy, Fantasy",https://cdn.myanimelist.net/images/anime/2/230...,Bandai Entertainment,20567.0,.hack//Gift,.hack//GIFT,CyberConnect2,R+ - Mild Nudity,6.1,Bee Train,As an expression of gratitude for the heroes o...,454.0,"Isshin Chiba, Shin'ya Kawatsura, Sayaka Aida, ...",This is a list of media related to the .hack m...,"anime animation,parody comedy,spoof,hot spring...",tt0823406,https://m.media-amazon.com/images/M/MV5BNjUwMT...,Helba has prepared a special hot springs for s...,This is a list of media related to the .hack m...
4,.hack//SIGN,tt0361140,None,None,.hack//Sign,"Adventure, Fantasy, Mystery",None,None,NaN,.hack//Sign,.hack//SIGN,None,None,6.95,None,"A young wavemaster, only known by the alias of...",48.0,"Amanda Winn Lee, Kaori Nazuka, Donna Rawlins, ...",.hack//Sign (stylized as .hack//SIGN) is a Jap...,"isekai,anime animation,female nudity,multiplay...",tt0361140,https://m.media-amazon.com/images/M/MV5BOTE5YW...,A strange boy named Tsukasa appears in the onl...,.hack//Sign (stylized as .hack//SIGN) is a Jap...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5177,I've Always Liked You,tt5889338,None,None,I've Always Liked You,Romance,None,None,NaN,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,None,None,7.22,None,Love is blooming at Sakuragaoka High School. N...,31245.0,"Kana Asumi, Gero, Yoshimi Narita, Momo Asakura...",Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...,"love,anime animation",tt5889338,https://m.media-amazon.com/images/M/MV5BOTcxNz...,"A girl finally confessed her love, but she ret...",Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...
5178,エクスドライバー Nina&Rei Danger Zone,tt1086281,None,None,UNKNOWN,"Action, Sci-Fi",None,None,NaN,eX-Driver: Nina & Rei Danger Zone,エクスドライバー Nina&Rei Danger Zone,None,None,6.39,None,The story takes place on a world 100 years in ...,1733.0,"Wataru Tanaka, Michiko Neya, Mami Kingetsu, Kô...","éX-Driver (エクスドライバー, Ekusudoraibā) is an anime...",female pedal pumping,tt1086281,https://m.media-amazon.com/images/M/MV5BZDlkOD...,A mysterious Mini-Ai-

In [252]:
fin_df2['text'] = fin_df2['Summary'].fillna(fin_df2['plot'])


In [245]:
fin_df2['text'] = fin_df2['text'].replace(np.nan, 'none')

In [248]:
fin_df2['text'][655]

'A slain detective is transformed by a scientific genius into a shape-shifting cyborg superhero who fights crime.'

In [255]:
fin_df2['text'] = fin_df2['text'].fillna(fin_df2['description'])

In [282]:
final_df.columns

Index(['imdb_title_title', 'imdb_title_titleId', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Genres',
       'anime_Image URL', 'anime_Licensors', 'anime_Members', 'anime_Name',
       'anime_Other name', 'anime_Producers', 'anime_Rating', 'anime_Score',
       'anime_Studios', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords', 'imdb_id', 'image',
       'description', 'text'],
      dtype='object')

In [286]:
fin_df3 = fin_df2.rename(columns = {'Title': 'imdb_title_title', 'imdb_id': 'imdb_title_titleId', 'Year': 'anime_Aired',
                            'duration':'anime_Duration', 'Genre': 'anime_Genres', 'content_rating': 'anime_Rating',
                            'rating_value': 'anime_Score', 'description': 'anime_Synopsis'})


In [294]:
fin_df3['anime_Image URL'] = 'none'
fin_df3['anime_Licensors'] = 'none'
fin_df3['anime_Producers'] = 'none'
fin_df3['anime_Studios'] = 'none'
fin_df3['anime_anime_id'] = 'none'
fin_df3['imdb_name_basics_primaryName'] = fin_df3['imdb_title_title']
fin_df3['anime_English name'] = fin_df3['imdb_title_title']
fin_df3['anime_Other name'] = fin_df3['imdb_title_title']

In [296]:
df_fin2 = fin_df3[['imdb_title_title', 'imdb_title_titleId', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Genres',
       'anime_Image URL', 'anime_Licensors', 
       'anime_Other name', 'anime_Producers', 'anime_Rating', 'anime_Score',
       'anime_Studios', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords',  'image', 'text']]

In [299]:
df_fin = final_df[['imdb_title_title', 'imdb_title_titleId', 'anime_Aired',
       'anime_Duration', 'anime_English name', 'anime_Genres',
       'anime_Image URL', 'anime_Licensors', 
       'anime_Other name', 'anime_Producers', 'anime_Rating', 'anime_Score',
       'anime_Studios', 'anime_Synopsis', 'anime_anime_id',
       'imdb_name_basics_primaryName', 'plot', 'keywords',  'image', 'text']]

In [316]:
df_fin['anime_anime_id'] = df_fin['anime_anime_id'].apply(lambda x: int(x))

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_97478/1803505878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin['anime_anime_id'] = df_fin['anime_anime_id'].apply(lambda x: int(x))


In [317]:
df_final_anime = pd.concat([df_fin,df_fin2],ignore_index = True)

In [334]:
df_final_anime.loc[df_final_anime['text'] == 'description', 'text'] = df_final_anime['anime_Synopsis']

In [321]:
import pickle

# File path to save the dictionary
file_path = "wiki_dict.pkl"

# Save the dictionary using pickle
with open(file_path, 'wb') as f:
    pickle.dump(combined_dict, f)


In [338]:
# Define a function to compare the lengths of 'text' and 'anime_Synopsis' columns and return the appropriate value
def replace_text(row):
    if len(row['text']) < len(row['anime_Synopsis']):
        return row['anime_Synopsis']
    else:
        return row['text']


In [339]:
df_final_anime['text'] = df_final_anime.apply(replace_text, axis=1)

In [343]:
df_final_anime['text'] = df_final_anime['text'].str.replace('== Plot ==', '')

In [345]:
df_final_anime.to_csv("combined_final_anime.csv")